In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Entitlements based on access metadata

Demonstrates the use of access metadata to grant access to portfolios in LUSID.

Attributes
----------
entitlements
portfolios
access metadata
"""

toggle_code("Toggle Docstring")

# Entitlements using access metadata

This notebook demonstrates the use of access metadata to grant access to portfolios in LUSID.

Table of contents:
1. [Setup](#1.-Setup)
2. [Create custom data type](#2.-Create-custom-data-type)
3. [Create custom property definition](#3.-Create-custom-property-definition)
4. [Create portfolio with custom property](#4.-Create-portfolio-with-custom-property)
5. [Set up entitlements policy](#5.-Set-up-entitlements-policy) <br>
    5.1. [Create a policy for the PortfolioStatus property](#4.1.-Create-a-policy-for-the-PortfolioStatus-property) <br>
    5.2. [Create a policy for the PortfolioStatus metadata](#4.2.-Create-a-policy-for-the-PortfolioStatus-metadata) <br>
    5.3. [Add the policies to a role](#4.3.-Add-the-policies-to-a-role) <br>
    5.4. [Assign the role to a user](#5.4.-Assign-the-role-to-a-user)
6. [Try to access portfolio (fail)](#6.-Try-to-access-portfolio-(fail))
7. [Update PortfolioStatus property and metadata](#7.-Update-PortfolioStatus-property-and-metadata)
8. [Try to access portfolio (success)](#8.-Try-to-access-portfolio-(success))

----


## 1. Setup

To start, let's import the libraries and initialise the APIs we'll use in the notebook.

In [2]:
# Import Libraries
import os
import json
import pytz
import pandas as pd
from datetime import datetime, timedelta
from IPython.core.display import HTML

import lusid
from lusid import models as models
import finbourne_access
from finbourne_access import models as access_models
import finbourne_identity
from finbourne_identity import models as identity_models
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_client = lusid_api_factory.api_client

lusid_api_url = api_client.configuration.host
access_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "access"
identity_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "identity"

access_api_factory = finbourne_access.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    access_url=access_api_url,
    app_name="LusidJupyterNotebook",
)

identity_api_factory = finbourne_identity.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    api_url=identity_api_url,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    lusid_api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.7270.0,0.5.2218,"{'relation': 'RequestLogs', 'href': 'http://de..."


In [3]:
# Initialise all APIs used in the notebook
data_types_api = lusid_api_factory.build(lusid.DataTypesApi)
properties_api = lusid_api_factory.build(lusid.PropertyDefinitionsApi)
transaction_portfolios_api = lusid_api_factory.build(lusid.TransactionPortfoliosApi)
portfolios_api = lusid_api_factory.build(lusid.PortfoliosApi)
policies_api = access_api_factory.build(finbourne_access.PoliciesApi)
access_roles_api = access_api_factory.build(finbourne_access.RolesApi)
identity_roles_api = identity_api_factory.build(finbourne_identity.RolesApi)

In [4]:
# Set key variables for the notebook
scope = "AccessMetadataEntitlements"
portfolio_code = "EntitledPortfolio"
portfolio_name = "Entitled Portfolio"
portfolio_status_data_type_code = "PortfolioStatusCodes"
portfolio_status_data_type_values = [
    "Approved",
    "Pending"
]
portfolio_status_property_code = "PortfolioStatus"
effective_date = "2021-01-01"
approved_portfolio_policy_code = "approved-portfolios-only"
property_policy_code = "allow-portfolio-status-property"
standard_lusid_access_policy_code = "allow-standard-lusid-features-access"

## 2. Create custom data type
We want to define a strict custom data type for use in this notebook, i.e. one that can only be `Approved` or `Pending`.

In [5]:
try:
    create_request = models.CreateDataTypeRequest(
        scope=scope,
        code=portfolio_status_data_type_code,
        type_value_range="Closed",
        display_name=f"Available {portfolio_status_data_type_code}",
        description=f"List of allowable values for {portfolio_status_data_type_code}",
        value_type="String",
        acceptable_values=portfolio_status_data_type_values
    )

    response = data_types_api.create_data_type(
        create_data_type_request=create_request
    )

    display(f"Data Type of {portfolio_status_data_type_code} has been created.")
    display(f"The acceptable values for this data type are: {str(portfolio_status_data_type_values)}")

except:
    response = data_types_api.get_data_type(
        scope=scope,
        code=portfolio_status_data_type_code
    )

    display(response.description)
    display(response.acceptable_values)

'List of allowable values for PortfolioStatusCodes'

['Pending', 'Approved']

## 3. Create custom property definition
We need to create a custom property which we will use to display the status of the portfolio. It will be defined using
the custom data type created in step #2.

In [6]:
try:
    properties_api.create_property_definition(
        create_property_definition_request=models.CreatePropertyDefinitionRequest(
            domain="Portfolio",
            scope=scope,
            code=portfolio_status_property_code,
            display_name=portfolio_status_property_code,
            data_type_id=models.ResourceId(
                code=portfolio_status_data_type_code,
                scope=scope
            )
        )
    )

except lusid.ApiException as e:
    display(json.loads(e.body)["title"])

## 4. Create portfolio with custom property

Create a new transaction portfolio with the property `PortfolioStatus` and value `Pending`.


In [7]:
try:
    request = transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_name,
            code=portfolio_code,
            base_currency="GBP",
            created=effective_date,
            sub_holding_keys=[],
            properties={
                f"Portfolio/{scope}/{portfolio_status_property_code}": models.ModelProperty(
                    key=f"Portfolio/{scope}/{portfolio_status_property_code}",
                    value=models.PropertyValue(
                        label_value="Pending"
                    )
                )
            }
        )
    )

except lusid.ApiException as e:
    display(json.loads(e.body)["title"])

## 5. Set up entitlements policies

We want to create an entitlement policy that grants access to portfolios, only if they are marked with a metadata key
`PortfolioStatus` of `Approved`.

We also want to create an entitlement policy that grants the user access to the portfolio property that manually
shadows the metadata key value of `Pending` or `Approved`.

We will need to add these policies to a role that can be assigned to a user.

To do this we will have to: <br>
5.1. [Create a policy for the PortfolioStatus property](#5.1.-Create-a-policy-for-the-PortfolioStatus-property)<br>
5.2. [Create a policy for the PortfolioStatus metadata](#5.2.-Create-a-policy-for-the-PortfolioStatus-metadata)<br>
5.3. [Add the policies to a role](#5.3.-Add-the-policies-to-a-role) <br>
5.4. [Assign the role to a user](#5.4.-Assign-the-role-to-a-user)

In [8]:
# WhenSpec objects specify the "lifetime" of a modification:
# when it is activated and when it is deactivated.
when_spec = access_models.WhenSpec(
    activate=datetime.now(tz=pytz.utc) - timedelta(days=2),
    deactivate=datetime(9999, 12, 31, tzinfo=pytz.utc),
)

### 5.1 Create a policy for the PortfolioStatus property

This policy should allow `Read` access to the `PropertyValue` and `PropertyDefinition` for property
`Portfolio/AccessMetadataEntitlements/PortfolioStatus`.

In [9]:
# Create the property policy using the policies api
try:
    policies_api.create_policy(
        access_models.PolicyCreationRequest(
            code=property_policy_code,
            applications=["LUSID"],
            grant=access_models.Grant.ALLOW,
            selectors=[access_models.SelectorDefinition(
                id_selector_definition=access_models.IdSelectorDefinition(
                    identifier={
                        "scope": scope,
                        "code": portfolio_status_property_code,
                        "domain": "Portfolio"
                    },
                    actions=[
                        access_models.ActionId(
                            scope="default",
                            activity="Read",
                            entity="PropertyValue"
                        ),
                        access_models.ActionId(
                            scope="default",
                            activity="Read",
                            entity="PropertyDefinition"
                        )
                    ]
                )
            )],
            when=when_spec
        )
    )
except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 613: # PolicyWithCodeAlreadyExists
        raise e

### 5.2 Create a policy for the PortfolioStatus metadata

This policy should allow `Read` and `List` access to any portfolio where the metadata key `PortfolioStatus` == `Approved`.


In [10]:
# Create the metadata policy using the policies api
try:
    policies_api.create_policy(
        access_models.PolicyCreationRequest(
            code=approved_portfolio_policy_code,
            applications=["LUSID"],
            grant=access_models.Grant.ALLOW,
            selectors=[access_models.SelectorDefinition(
                metadata_selector_definition=access_models.MetadataSelectorDefinition(
                    expressions=[
                        access_models.MetadataExpression(
                            metadata_key=portfolio_status_property_code,
                            operator=access_models.Operator.EQUALS,
                            text_value="Approved"
                        )
                    ],
                    actions=[
                        access_models.ActionId(
                            scope="default",
                            activity="Read",
                            entity="Portfolio"
                        ),
                        access_models.ActionId(
                            scope="default",
                            activity="List",
                            entity="Portfolio"
                        )
                    ]
                )
            )],
            when=when_spec
        )
    )
except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 613: # PolicyWithCodeAlreadyExists
        raise e

### 5.3 Add the policies to a role

One implementation detail for LUSID roles is that we'll have to create the same role twice: once using the identity API
and once using the access API. This is such that the access module, which handles applying policies to a role,
can communicate with the identity module, which handles applying roles to users.

In [11]:
# Create the role using the access API
try:
    access_roles_api.create_role(
        role_creation_request=access_models.RoleCreationRequest(
            code=approved_portfolio_policy_code,
            description=approved_portfolio_policy_code,
            resource=access_models.RoleResourceRequest(
                policy_id_role_resource=access_models.PolicyIdRoleResource(
                    # Here we apply the policy we defined earlier as well as a default policy to provide basic access
                    policies=[
                        access_models.PolicyId(
                            scope="default",
                            code=standard_lusid_access_policy_code
                        ),
                        access_models.PolicyId(
                            scope="default",
                            code=approved_portfolio_policy_code
                        ),
                        access_models.PolicyId(
                            scope="default",
                            code=property_policy_code
                        )
                    ]
                )
            ),
            when=when_spec
        )
    )

except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 613: # Role with code already exists
        raise e


# Create the same role using the identity API
try:
    identity_roles_api.create_role(
        create_role_request=identity_models.CreateRoleRequest(
            name=approved_portfolio_policy_code
        )
    )

except finbourne_identity.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 157: # Role with code already exists
        raise e

### 5.4. Assign the role to a user

Whilst logged into the LUSID UI as a user with entitlements to assign roles to other users,
assign this role to a user with no other roles or policy entitlements.

----

## 6. Try to access portfolio (fail)

In the LUSID UI whilst logged in as the new user, assigned only with the newly created role, try to load or view the
details for the portfolio created in this notebook.

This should fail as the user should not have access to the portfolio due to there being no `PortfolioStatus` metadata
key with a value of `Approved`.

----

## 7. Update PortfolioStatus property and metadata

Now update the `PortfolioStatus` property and metadata to `Approved`, signifying that the administrator has approved
the portfolio for read access for users assigned to the new custom role.

These two updates can be wrapped in one function so we can easily pass in `Approved` or `Pending` to switch between the two statuses.

In [12]:
def update_portfolio_status(status):
    if status not in portfolio_status_data_type_values:
        return print(f"'{status}' is not one of the approved values: {portfolio_status_data_type_values}")
    
    # Update property
    portfolios_api.upsert_portfolio_properties(
        scope=scope,
        code=portfolio_code,
        request_body={
            f"Portfolio/{scope}/{portfolio_status_property_code}": models.ModelProperty(
                key=f"Portfolio/{scope}/{portfolio_status_property_code}",
                value=models.PropertyValue(
                    label_value=status
                )
            )
        }
    )

    # Access metadata
    portfolios_api.upsert_portfolio_access_metadata(
        scope=scope,
        code=portfolio_code,
        metadata_key=portfolio_status_property_code,
        upsert_portfolio_access_metadata_request=models.UpsertPortfolioAccessMetadataRequest(
            metadata=[
                models.AccessMetadataValue(
                    value=status
                )
            ]
        )
    )
    
    print(f"PortfolioStatus updated to '{status}'")

In [13]:
# Update the status
update_portfolio_status("Approved")

PortfolioStatus updated to 'Approved'


----

## 8. Try to access portfolio (success)

In the LUSID UI whilst logged in as the new user, assigned only with the newly created role, try to load or view the
details for the portfolio created in this notebook.

This should now pass as the user should have access to the portfolio due to the `PortfolioStatus` metadata
key now having a value of `Approved`.